# Portfolio Management using the Efficient Frontier


In this project, we will conduct a top-down analysis; however, macroeconomic factors such as the inflation rate will be omitted for convenience. Hence, we will proceed through the following pipeline:

1. Perform an industry analysis based on the sector ETFs' performance over the past three years.
2. Select sectors that have the following traits for further investigation of assets within those sectors:
    - Exhibit positive risk-adjusted performance 
    - Low correlation between asset classes  
3. If investing in an individual asset rather than a sector ETF yields a better risk-adjusted return, substitute the ETF with the individual asset in the portfolio.
4. Optimize the efficient frontier to determine the optimal weights of the asset classes.


Only sector indexes will be used in portfolio management, including:

- XLE: Energy Select Sector SPDR Fund
    - EXXON MOBIL CORP (XOM)
    - CHEVRON CORP (CVX)
- XLF: Financial Select Sector SPDR Fun
    - Berkshire Hathaway Inc. Class B (BRK.B)
    - JPMorgan Chase & Co. (JPM)
- XLU: Utilities Select Sector SPDR Fund
    - Exelon Corp (EXC)
    - Xcel Energy Inc (XEL)
- XLK: Technology Select Sector SPDR Fund
    - Microsoft Corporation (MSFT)
    - Apple Inc. (AAPL)
- XLV: Health Care Select Sector SPDR Fund
    - Pfizer Inc. (PFE)
    - Thermo Fisher Scientific Inc. (TMO)
- XLY: Consumer Discretionary Select Sector SPDR Fund
    - Amazon.com Inc (AMZN)
    - Home Depot Inc (HD)
- XLP: Consumer Staples Select Sector SPDR Fund
    - COCA COLA CO (KO)
    - COSTCO WHOLESALE CORP	(COST)
- GLD: SPDR Gold Shares





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Github_Commit/MPT

In [ ]:
!git config --global user.email 'victorius918@gmail.com'
!git config --global user.name 'HyunminHong'

In [ ]:
!git add Sector_Portfolio.ipynb

In [ ]:
!git commit -m 'MPT functionalise dirty codes'
!git push

In [ ]:
import yfinance as yf
from pandas_datareader import data as pdr
import scipy.stats
from sklearn.linear_model import LinearRegression
import datetime as dt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from pylab import mpl, plt 
plt.style.use('seaborn') 
mpl.rcParams['font.family'] = 'serif' 
%matplotlib inline
yf.pdr_override()

# Asset Returns
$$
R_t = \frac{P_t - P_{t-1}}{P_{t-1}}\approx \log(1 + P_t),\quad\quad \log R_t = \log P_t - \log P_{t-1}
$$

# Sharpe ratio vs. Sortino ratio

- **Sharpe ratio** is preferred to evaluate low-volatility investment portfolios
- **Sortino ratio** is preferred to evaluate high-volatility investment portfolios

In [ ]:
def extract_data(ticker, column=None, asset_class='equity', observation_period=3, startDate=None, endDate=None):
    """
    Parameters
        ticker: input tickers 
            'GS5' is a 5-year treasury bond
            'GS10' is a 10-year treasury bond
        column: if a user wants to change the column names, input the desired names
        type: 
            'equity' if one wants to extract equity data
            'bond' if one wants to extract bond data
        observation_period: the number of years of the panel data
        startDate: starting date of the panel data
        endDate: ending date of the panel data
    Returns
        df: panel data pd.DataFrame object
    """
    if endDate is None:
        endDate = dt.datetime.now()
        startDate = endDate - dt.timedelta(days=365*observation_period)
    
    if asset_class == 'equity':
        df = pdr.get_data_yahoo(ticker, start=startDate, end=endDate)['Adj Close']
        if column is None:
            return df
        else:
            df.columns = column
            return df

    if asset_class == 'bond':
        return pdr.get_data_fred(ticker, start=startDate, end=endDate)

    
def asset_return(df, rtn_type='log'):
    """
    Parameters
        df: input the panel data that we want to extract the return values
        rtn_type: choose the return type between 'log' or arithmatic'. Computational feasibility wise
    Returns
        rtn: asset return data pd.DateFrame object
    """
    if rtn_type == 'arithmatic': 
        rtn = df / df.shift(1) - 1
        rtn.dropna(axis=0, how='any', inplace=True)
    elif rtn_type == 'log': 
        rtn = np.log(df) - np.log(df.shift(1))
        rtn.dropna(axis=0, how='any', inplace=True)
    else:
        raise ValueError('Return type is not valid.')
    return rtn


def asset_statistics(rtn, rtn_market, trading_period='daily'):
    """
    Parameters
        rtn: asset returns that we want to analyse
        rtn_market: market returns based on S&P 500
        trading_period: time period of the given time series data, e.g., 'daily', 'weekly', 'monthly'
    Returns
        stats: market information statistics, e.g., market beta, volatility, Sharpe ratio
    """
    if trading_period == 'daily':
        trading_num = 252
    elif trading_period == 'weekly':
        trading_num = 52
    elif trading_period == 'monthly':
        trading_num = 12
    
    stats = pd.DataFrame()

    # Beta computes the market beta of each asset class
    for col in rtn.columns:
        beta = np.cov(rtn[col], rtn_market)[0][1] / np.var(rtn_market)
        stats.loc[col, 'Beta'] = beta.round(3)

    # MDD computation requires to know the previous peaks
    drawdown = (df_sector - df_sector.cummax()) / df_sector.cummax()
    MDD = -(drawdown.min())*100

    # We use log returns instead of returns for convenience
    returns_annual = rtn_sector.mean()*trading_num*100
    volatility_annual = rtn_sector.std(ddof=0)*np.sqrt(trading_num)*100
    semi_std = rtn_sector[rtn_sector < 0].std(ddof=0)*np.sqrt(trading_num)*100

    stats['Annualized Returns(%)'] = returns_annual.round(3)
    stats['Annualized Volatility(%)'] = volatility_annual.round(3)
    stats['Sharpe Ratio'] = (np.subtract(returns_annual, float(Rf))/volatility_annual)
    stats['Sortino Ratio'] = (np.subtract(returns_annual, float(Rf))/semi_std) # preferred to evaluate high-volatility portfolios
    stats['Maximum Drawdown(%)'] = MDD.round(3)
    stats['Skewness'] = scipy.stats.skew(rtn_sector)
    stats['Excess Kurtosis'] = scipy.stats.kurtosis(rtn_sector)
    return stats


def is_normal(rtn, level=0.01, method='shapiro wilk'):
    """
    Using Jarque Bera test to determine whether the given returns are normally distributed or not.
    H0: the data in vector x comes from a normal distribution with an unknown mean and variance
    H1: the data is not from such distribution

    Parameters
        rtn: asset returns that we want to analyse
        level: confidence level, alpha
    Returns
        True if the given returns are normally distributed
        False if the given returns are not normally distributed
    """
    if method == 'jarque bera':
        testStat, pVal = scipy.stats.jarque_bera(rtn)
    if method == 'shapiro wilk':
        testStat, pVal = scipy.stats.shapiro(rtn)
    return pVal > level

In [ ]:
# Use the following sector ETFs 
tickers_sector = ['XLE', 'XLF', 'XLU', 'XLK', 'XLV', 'XLY', 'XLP', 'GLD']

# Use SPY as the market 
tickers_market = ['SPY']

# Converts DatetimeArray/Index to PeriodArray/Index.
#df.index = df.index.to_period('W') 
columns_sector = ['Gold', 'Energy', 'Financial', 'Technology', 'Staples', 'Utilities', 'Health Care', 'Discretionary']
columns_market = ['Market']

In [ ]:
df_sector = extract_data(ticker=tickers_sector, column=columns_sector)
df_market = extract_data(ticker=['SPY'])

# Use 5-year U.S. Treasury yield as a proxy for the risk-free rate
Rf = float(extract_data('GS5', asset_class='bond').mean())

# Find the return of each asset class as it is more informativ than closing price
rtn_sector = asset_return(df_sector)
rtn_market = asset_return(df_market)

In [ ]:
stats = asset_statistics(rtn_sector, rtn_market)
stats

In [ ]:
# Define the x-axis positions for the bars
pos = np.arange(len(stats))
bar_width = 0.35

# Plot the 'Sharpe ratio' bars in blue
plt.bar(pos, stats['Sharpe Ratio'].sort_values(), width=bar_width, label='Sharpe ratio')

# Plot the 'Sortino ratio' bars in red
plt.bar(pos + bar_width, stats['Sortino Ratio'].sort_values(), width=bar_width, label='Sortino ratio')

# Set the x-axis tick labels
plt.xticks(pos + bar_width / 2, stats['Sortino Ratio'].sort_values().index, rotation=45)

# Set the title and labels
plt.title('Sector Ratios for current {} years'.format(3))
plt.xlabel('Sectors')
plt.ylabel('Ratio Values')

# Add horizontal lines
plt.axhline(y=0.4, color='blue', linestyle='--', label='Sharpe Ratio = 0.4')
plt.axhline(y=0.6, color='green', linestyle='--', label='Sortino Ratio = 0.6')
plt.legend()

When the Sharpe ratio greater than 0.4 and Sortino ratio greater than 0.6 is considered worth to invest, only four following sectors passed the threshold:
1. Energy
2. Techonology
3. Staples
4. Health Care
5. Financial

However, it's crucial to carefully evaluate the risk-reward tradeoff, consider the investment objectives, time horizon, and risk tolerance before making a decision. Hence, we will, for now, keep other non-passed sectors to the next step. 

# Correlations between Sectors



In [ ]:
plt.figure(figsize=(16, 6))

# Compute the correlation matrix
corr_sector = rtn_sector.corr()

# Create a mask to hide the upper triangle of the correlation matrix
mask = np.triu(np.ones_like(corr_sector, dtype=bool))

# Create a triangular correlation heatmap
heatmap = sns.heatmap(corr_sector, mask=mask, annot=True, cmap='BrBG',
                      vmin=-1, vmax=1, square=True)

# Set the title and axis labels
heatmap.set_title('Correlation Heatmap between Sectors', fontdict={'fontsize': 18}, pad=16)
plt.xlabel('Asset Classes')
plt.ylabel('Asset Classes')

# Rotate the tick labels for better visibility
plt.xticks(rotation=45)
plt.yticks(rotation=0)

Correlation analysis measures the degree of relationship between two variables. By conducting correlation analysis, we can determine the extent of the relationship between two asset classes and assess their level of correlation. According to Modern Portfolio Theory (MPT), investors generally benefit more from diversification when there is a low correlation between two asset classes, while assets within the same asset class exhibit a high correlation.

Let us establish specific thresholds to ensure clarity before the correlation analysis.
* $0.8 \leq |\rho| \leq 1$. Very highly correlated
* $0.6 \leq |\rho| < 0.8$. Highly correlated
* $0.4 \leq |\rho| < 0.6$. Moderately correlated
* $0.2 \leq |\rho| < 0.4$. Weak correlation
* $0 \leq |\rho| < 0.2$. Very weak correlation / No correlation


According to the correlation heatmap above, **gold** and **energy** tend to show low positive correlations to other sectors, yet we observe that **energy** is highly correlated with **financial sectors**. 

**Technology** also shows low to moderate correlations, but it simultaneously exhibits a strong correlation between **discretionary**. When considering the Sharpe ratio of **discretionary** relative to **techonology**, **discretionary** is considered to not bring a high diversification benefit.

# Relative Performance by Sectors

In [ ]:
# Normalized prices to measure the relative performance by sectors
normPrices = df_sector / df_sector.iloc[0]
normPrices.plot(figsize=(8, 6))

In [ ]:
rtn_sector.aggregate(is_normal)

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(10, 10), sharey=True)
fig.suptitle('VaR measure for Sectors', fontsize=20)

const = 0
for i in range(len(axes)):
    for j in range(len(axes[0])):
        axes[i][j] = rtn_sector[rtn_sector.columns[const]].plot(kind='hist', bins=100, ax=axes[i][j])
        axes[i][j].axvline(np.percentile(rtn_sector[rtn_sector.columns[const]], 5), color='orange', label='5%')
        axes[i][j].axvline(np.percentile(rtn_sector[rtn_sector.columns[const]], 1), color='red', label='1%')
        axes[i][j].set_title('{} VaR '.format(rtn_sector.columns[const]), loc='left', fontsize=14)
        const += 1

plt.tight_layout() # Ensures proper spacing between subplots

# VaR and CVaR

- Historical VaR
- Parametric VaR
- Modified Conish-Fisher VaR

In [ ]:
def varHistoric(returns, alpha=5):
    """
    VaR Historic
    """
    if isinstance(returns, pd.DataFrame):
        return returns.aggregate(varHistoric, alpha=alpha)
    elif isinstance(returns, pd.Series):
        return -np.percentile(returns, alpha)
    else:
        raise TypeError("Not allowed data type")

def varGaussian(returns, alpha=5, modified=False):
    z = scipy.stats.norm.ppf(alpha/100)

    if modified:
        s = scipy.stats.skew(returns)
        k = scipy.stats.kurtosis(returns) + 3
        z = (z + (z**2 - 1)*s/6 + (z**3 - 3*z)*(k-3)/24 - (2*z**3 - 5*z)*(s**2)/36)
        
    return -(returns.mean() + z*returns.std(ddof=0))

def cvarHistoric(returns, alpha=5):
    """
    CVaR Historic
    """
    if isinstance(returns, pd.Series):
        is_beyond = returns <= -varHistoric(returns, alpha=alpha)
        return -returns[is_beyond].mean()
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(cvarHistoric, alpha=alpha)
    else:
        raise TypeError("Not allowed data type")

In [ ]:
var_list = [varGaussian(rtn), varGaussian(rtn, modified=True), varHistoric(rtn)]
comparision = pd.concat(var_list, axis=1)
comparision.columns = ['Gaussian', 'Cornish-Fisher', 'Historic']
comparision.plot.bar(title="VaR: Sectors")

# Efficient Frontier

In [ ]:
annualized_rtn = stats['Annualized Returns(%)']